In [1]:
import pickle
import json
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import wandb
from wandb.keras import WandbCallback

from tensorflow import keras
from pathlib import Path

%matplotlib inline

In [5]:
DATA = Path('./data')

In [ ]:
!wandb login

In [36]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar100.load_data(label_mode = 'fine')

In [37]:
assert x_train.shape == (50000, 32, 32, 3)
assert x_test.shape == (10000, 32, 32, 3)
assert y_train.shape == (50000, 1)
assert y_test.shape == (10000, 1)

In [3]:
# lets pickle the data so we can store it in a bucket
data = [x_train, y_train, x_test, y_test]
with open(DATA/'training_data.dat', "wb") as f:
    pickle.dump(data, f)

In [11]:
with open(DATA/'training_data.dat', "rb") as f:
    [x_train, y_train, x_test, y_test] = pickle.load(f)

In [12]:
assert x_train.shape == (50000, 32, 32, 3)
assert x_test.shape == (10000, 32, 32, 3)
assert y_train.shape == (50000, 1)
assert y_test.shape == (10000, 1)

In [ ]:
# from: https://colab.research.google.com/github/wandb/examples/blob/master/colabs/keras/Simple_Keras_Integration.ipynb#scrollTo=_Mqy_ZBNM8UA
run = wandb.init(project='mlops-midterm',
                 config={
                     "learning_rate": 0.005,
                     "epochs": 10,
                     "batch_size": 64,
                     "loss_function": "sparse_categorical_crossentropy",
                     "metrics":["sparse_categorical_accuracy"],
                     "optimizer": "adam",
                     "base": "EfficientNetB0",
                     "pool": "avg",
                     "head": "Dense(100)",
                     "dataset": "CIFAR-100"
                 })
config = wandb.config

In [10]:
dummy = np.random.randint(0,255,(1,32,32,3))

In [47]:
base = keras.applications.EfficientNetB0(
    include_top = False,
    weights = 'imagenet',
    input_shape = [32, 32, 3],
    pooling ='avg' #config.pool
)
base.trainable = False

Build and compile model

In [49]:
# use functional approach so gradcam model works
drop = keras.layers.Dropout(0.2)(base.output)
preds = keras.layers.Dense(100, activation = 'sigmoid')(drop)
model = keras.models.Model(inputs = base.inputs, outputs = preds)

In [57]:
# model.summary()

In [50]:
optimizer = tf.keras.optimizers.Adam() #config.learning_rate
model.compile(
    optimizer = optimizer,
    loss = 'sparse_categorical_crossentropy',
    metrics = ['acc']
)

In [51]:
dummy_out = model(np.random.randint(0,255,(1,32,32,3)))

In [52]:
# grad model for Grad-CAM
grad_model = keras.models.Model(
    [model.inputs],
    [model.get_layer('top_conv').output, model.output]
)

In [53]:
with tf.GradientTape() as tape:

    last_conv_layer_output, preds = grad_model(np.random.randint(0,255,(1,32,32,3)))

    pred_index = tf.argmax(preds[0])
    class_channel = preds[:, pred_index]
    
    # This is the gradient of the output neuron (top predicted or chosen)
    # with regard to the output feature map of the last conv layer
    grads = tape.gradient(class_channel, last_conv_layer_output)

In [54]:
class_channel

<tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.67721796], dtype=float32)>

In [55]:
type(grads)

tensorflow.python.framework.ops.EagerTensor

In [13]:
dummy_out = model(np.random.randint(0,255,(1,32,32,3)))

In [16]:
np.argmax(dummy_out)

14

In [4]:
optimizer = tf.keras.optimizers.Adam() #config.learning_rate
model.compile(
    optimizer = optimizer,
    loss = 'sparse_categorical_crossentropy',
    metrics = ['acc']
)

In [ ]:
model.fit(
    x = x_train,
    y = y_train,
    validation_data = (x_test, y_test),
    epochs = config.epochs,
    callbacks = [WandbCallback]
)

In [18]:
with open('cfg_00001.json', 'r') as f:
    config = json.load(f)

In [20]:
config

{'data_set': 'training_data.dat',
 'learning_rate': 0.005,
 'epochs': 10,
 'batch_size': 64,
 'loss_function': 'sparse_categorical_crossentropy',
 'metrics': ['sparse_categorical_accuracy'],
 'optimizer': 'adam',
 'base': 'EfficientNetB0',
 'pool': 'avg',
 'head': 'Dense(100)',
 'dataset': 'CIFAR-100'}

In [ ]:
options=tf.saved_model.SaveOptions(experimental_custom_gradients=True))

In [2]:
model = tf.keras.models.load_model('./model_test/')

2021-08-06 16:18:51.755875: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [31]:
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

In [38]:
dummy_out = model(x_train[:10,:,:,:])

In [39]:
dummy_out.shape

TensorShape([10, 100])

In [40]:
np.argmax(dummy_out, axis = 1)

array([19, 29,  0, 11,  1, 61, 90, 28, 23, 29])

In [41]:
y_train[:10,:]

array([[19],
       [29],
       [ 0],
       [11],
       [ 1],
       [86],
       [90],
       [28],
       [23],
       [31]])

In [43]:
x_train[0,0,0,:].dtype

dtype('uint8')